In [5]:
pip install xlsxwriter

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.2 MB/s eta 0:00:00.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import xlsxwriter

# Sample data
categories = ["Poor Service", "Late Deliveries", "Defective Products", "Incorrect Orders", "Others"]
frequencies = [35, 20, 15, 10, 20]

# Calculate cumulative frequencies and percentages
total = sum(frequencies)
cumulative_frequencies = [sum(frequencies[:i+1]) for i in range(len(frequencies))]
cumulative_percentages = [cf / total * 100 for cf in cumulative_frequencies]

# Create a DataFrame
df = pd.DataFrame({'Category': categories, 'Frequency': frequencies,
                   'Cumulative Frequency': cumulative_frequencies,
                   'Cumulative Percentage': cumulative_percentages})

# Create a new Excel workbook and add a worksheet
workbook = xlsxwriter.Workbook('InteractiveParetoChart.xlsx')
worksheet = workbook.add_worksheet()

# Write the data to the worksheet
worksheet.write_row('A1', ['Category', 'Frequency', 'Cumulative Frequency', 'Cumulative Percentage'])
worksheet.write_column('A2', df['Category'])
worksheet.write_column('B2', df['Frequency'])
worksheet.write_column('C2', df['Cumulative Frequency'])
worksheet.write_column('D2', df['Cumulative Percentage'])

# Add data validation for selecting categories
worksheet.data_validation('F2', {'validate': 'list',
                                 'source': '=Sheet1!$A$2:$A$6'})

# Add hyperlinks to each category cell to navigate to a specific cell in the worksheet
for i, category in enumerate(categories):
    worksheet.write_url(i + 1, 5, f'#\'Sheet1\'!A{i+2}', string=category)

# Create a chart object
chart = workbook.add_chart({'type': 'column'})

# Configure the series
chart.add_series({
    'categories': '=Sheet1!$A$2:$A$6',
    'values': '=Sheet1!$B$2:$B$6',
    'data_labels': {'value': True}
})

# Add a second series for the cumulative percentage
chart.add_series({
    'categories': '=Sheet1!$A$2:$A$6',
    'values': '=Sheet1!$D$2:$D$6',
    'y2_axis': True,
    'marker': {'type': 'none'}
})

# Set the chart title and axis labels
chart.set_title({'name': 'Pareto Chart'})
chart.set_x_axis({'name': 'Categories', 'text_axis': True, 'num_font': {'rotation': -90}})
chart.set_y_axis({'name': 'Frequency'})
chart.set_y2_axis({'name': 'Cumulative Percentage'})

# Insert the chart into the worksheet
worksheet.insert_chart('F2', chart)

# Close the workbook
workbook.close()
